In [1]:
import sys
import glob
import pickle
import hashlib
import PIL
from os.path import splitext, basename, join, isfile
from collections import defaultdict, Counter
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform

from delf import feature_io

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
filenames_index_ix = pickle_load('../submit/2_full_indexing/filenames_index_ix.pkl')
filenames_query_ix = pickle_load('../submit/2_full_indexing/filenames_query_ix.pkl')
filenames_index = pickle_load('../submit/2_full_indexing/filenames_index.pkl')
filenames_query = pickle_load('../submit/2_full_indexing/filenames_query.pkl')

In [ ]:
location_index = []
for _, nb in sorted(Counter(filenames_index_ix).items(), key=lambda x: x[0]):
    location_index.extend(np.arange(nb))

location_query = []
for _, nb in sorted(Counter(filenames_query_ix).items(), key=lambda x: x[0]):
    location_query.extend(np.arange(nb))

In [ ]:
I_mapped = pickle_load('../submit/2_full_indexing/mapped_index_compress.pkl')
I = pickle_load('../submit/2_full_indexing/index_compress.pkl')
# D = pickle_load('../submit/2_full_indexing/distance_compress.pkl')

In [ ]:
results = defaultdict(Counter)
dump = False
for i, ix in enumerate(filenames_query_ix):
    if ix != 25672:
        continue
    results[ix] += Counter(I_mapped[i])

# keys = sorted(list(results.keys()))
# for i, query_ix in enumerate(keys):
#     if query_ix > 0:
#         break
#     values = results[query_ix]
#     if query_ix in values.keys():
#         values.pop(query_ix)
#     values = sorted(values.items(), key=lambda x: x[1], reverse=True)[:100]
#     results[filenames_query[query_ix]] = ' '.join([filenames_index[ix] for ix, _ in values if ix != -1])
#     results.pop(query_ix)

In [88]:
index = np.random.randint(0, 200, size=(115000, 100))

In [33]:
filenames_index_ix = []
for i in range(int(115000 / 10)):
    filenames_index_ix.extend([i] * 10)

In [34]:
mapped_index = np.empty_like(index)
for i, j in np.ndindex(index.shape):
    ix = index[i, j]
    mapped_index[i, j] = filenames_index_ix[ix] if ix != -1 else -1

In [35]:
mapped_index

array([[ 8987, 10852,  2998, ...,  7651, 10262,  7033],
       [10712, 11177,  1068, ...,  2798,  9407,  5366],
       [10155,  8406,  8767, ...,  3989,  8435,  3225],
       ...,
       [10509,  9108,  6173, ...,  5975,  9870,  7928],
       [ 4958,  5829,  9103, ..., 10513,  9828,   172],
       [11041,  7172,  6277, ...,  1167,  1072,  6785]])

In [44]:
subset = index[:100, :]

In [47]:
subset[mapped_index[:100, :] == 8987]

array([-1, -1, -1])

In [84]:
%%timeit
value, count = np.unique(index[100:500], return_counts=True)
counter = Counter(dict(zip(value, count)))

1.12 ms ± 2.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [72]:
%%timeit
counter = Counter()
for i in range(100, 500):
    counter += Counter(index[i])

16.2 ms ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [89]:
counter1 = Counter()
for i in range(100, 500):
    counter1 += Counter(index[i])

value, count = np.unique(index[100:500], return_counts=True)
counter2 = Counter(dict(zip(value, count))).most_common(100)

In [86]:
counter1 == counter2

True

In [90]:
counter2

[(11, 242),
 (187, 234),
 (61, 231),
 (9, 224),
 (67, 224),
 (75, 224),
 (27, 223),
 (80, 223),
 (145, 223),
 (4, 221),
 (32, 221),
 (167, 221),
 (191, 221),
 (198, 221),
 (52, 220),
 (93, 219),
 (174, 219),
 (48, 218),
 (55, 218),
 (86, 218),
 (41, 217),
 (72, 217),
 (150, 217),
 (115, 216),
 (156, 216),
 (108, 215),
 (161, 214),
 (163, 214),
 (178, 214),
 (30, 213),
 (66, 213),
 (122, 213),
 (143, 213),
 (196, 213),
 (190, 212),
 (1, 211),
 (33, 211),
 (90, 211),
 (120, 211),
 (154, 211),
 (173, 211),
 (14, 210),
 (68, 210),
 (138, 210),
 (155, 210),
 (177, 210),
 (57, 209),
 (126, 209),
 (164, 209),
 (172, 209),
 (2, 208),
 (13, 208),
 (77, 208),
 (103, 208),
 (166, 208),
 (183, 208),
 (7, 207),
 (10, 207),
 (23, 207),
 (42, 207),
 (50, 207),
 (62, 207),
 (109, 207),
 (111, 207),
 (117, 207),
 (147, 207),
 (18, 206),
 (51, 206),
 (59, 206),
 (83, 206),
 (91, 206),
 (132, 206),
 (176, 205),
 (5, 204),
 (54, 204),
 (74, 204),
 (186, 204),
 (8, 203),
 (40, 203),
 (79, 203),
 (104, 203)